# Interactions with Bokeh

In this notebook we will build on our previous work and add a variety of interactions to the plots that we have created.

### Data

Let's begin by loading the data that we will need for our visualizations.

In [1]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np

# SPY Close, Returns, Realized Volatility, and VIX
df_spy = pdr.get_data_yahoo(['SPY', '^VIX'], start='2016-01-01', end='2021-09-30')
df_spy = df_spy.round(2)
df_spy = df_spy['Close'].reset_index()
df_spy.rename(columns={'Date':'date','SPY':'close','^VIX':'vix'}, inplace=True)
df_spy['return'] = np.log(df_spy['close'] / df_spy['close'].shift(1))
df_spy['realized_vol'] = df_spy['return'].rolling(42).std() * np.sqrt(252)

# SPY Monthly Returns
df_spy['year'] = df_spy['date'].dt.year
df_spy['month'] = df_spy['date'].dt.month
df_monthly = df_spy.groupby(['year', 'month'], as_index=False)[['return']].sum()
df_monthly['year_month'] = (df_monthly['year'] * 100) + df_monthly['month']
df_monthly['year_month'] = df_monthly['year_month'].astype(str)

# Implied Leverage Effect
df_spy['vix_change'] = df_spy['vix'].diff()
df_spy['vix'] = df_spy['vix'] / 100
df_spy['vix_change'] = df_spy['vix_change'] / 100

# Asset Allocation
df_asset_allocation = pd.read_csv('asset_allocation.csv', parse_dates=['trade_date'])

### Linked Panning with `gridplot()`

In the previous notebook, we created a `column()` of plots to visualize the leverage effect.  However, the plots were all independent of one another, which may not be desireable.  

We remedy that here using the `gridplot()` function.  We also modify the `x_range` inputs of all of our figures so that they link properly.

Notice that we now have a single toolbar for all four plots.

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import NumeralTickFormatter

output_notebook()

# defining plot options all at once
plot_options = dict(width=600, height=200, x_axis_type='datetime')

# defining multiple plots - notice the change in the x_range for p2, p3, p3
p1 = figure(**plot_options, title='SPY Leverage Effect')
p1.line(x=df_spy['date'], y=df_spy['close'], legend_label='close')
p1.legend.location = 'top_left'

p2 = figure(x_range=p1.x_range,  **plot_options)
p2.line(x=df_spy['date'], y=df_spy['return'], legend_label='return')
p2.yaxis.formatter = NumeralTickFormatter(format='0%')
p2.legend.location = 'top_left'

p3 = figure(x_range=p1.x_range, **plot_options)
p3.line(x=df_spy['date'], y=df_spy['realized_vol'], legend_label='realized volatility')
p3.yaxis.formatter = NumeralTickFormatter(format='0%')
p3.legend.location = 'top_left'

p4 = figure(x_range=p1.x_range, **plot_options)
p4.line(x=df_spy['date'], y=df_spy['vix'], legend_label='vix')
p4.yaxis.formatter = NumeralTickFormatter(format='0%')
p4.legend.location = 'top_left'

# putting all plots into a grid plot
p = gridplot([[p1], 
              [p2], 
              [p3],
              [p4]]
            )

show(p)

Loading BokehJS ...

### Specifying Tools

We can also specify the interactive tools that we want in our graph.

The easiest way to do this is the `tools` argument of the `figure()` function which takes values that are comma delimited strings such as `'reset,hover,save'`.

In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models import NumeralTickFormatter

output_notebook()

cds = ColumnDataSource(df_spy)

# creating custom tool list
tools = 'box_select,box_zoom,lasso_select,pan,wheel_zoom,reset,hover,save'

# adding custom tool list to figurw
p = figure(plot_width=600, plot_height=400, tools=tools, title='SPY Implied Leverage Effect',
           x_axis_label='return', y_axis_label='vix change')

# adding glyph
p.hex('return', 'vix_change', line_color="navy", fill_color="orange", size=15, source=cds)

# formatting the x-axis and y-axis to percents
p.xaxis.formatter = NumeralTickFormatter(format='0%') 
p.yaxis.formatter = NumeralTickFormatter(format='0%')

show(p)

Loading BokehJS ...

### Linked Properties

We can also link properities of the graph to widgets.

The example below allows us to modify the size of the `.hex` glyphs so we can dial in the appearance of our graph.

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models import Slider
from bokeh.layouts import column
from bokeh.models import NumeralTickFormatter

output_notebook()

cds = ColumnDataSource(df_spy)

p = figure(plot_width=600, plot_height=400, title='SPY Implied Leverage Effect',
           x_axis_label='return', y_axis_label='vix change')

# adding glyph
r = p.hex('return', 'vix_change', line_color="navy", fill_color="orange", size=10, source=cds)

# linking slider to  the size of the hex glyph 
slider = Slider(start=1, end=20, step=1, value=10)
slider.js_link('value', r.glyph, 'size')

# formatting the x-axis and y-axis to percents
p.xaxis.formatter = NumeralTickFormatter(format='0%') 
p.yaxis.formatter = NumeralTickFormatter(format='0%')

show(column(p, slider))

Loading BokehJS ...

### Linked Brushing

Linked brushing allows us to highlight related data between two graphs.

In [5]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import NumeralTickFormatter

output_notebook()

cds = ColumnDataSource(df_spy)

# defining plot options all at once, passing custom tool list directly
plot_options = dict(width=500, height=350, tools='box_zoom,lasso_select,reset, box_select')

p1 = figure(**plot_options, title='SPY Leverage Effect', x_axis_label='return', y_axis_label='vix change')
p1.hex('return', 'vix_change', line_color="navy", fill_color="orange", size=10, source=cds)
# formatting the x-axis and y-axis to percents
p1.xaxis.formatter = NumeralTickFormatter(format='0%') 
p1.yaxis.formatter = NumeralTickFormatter(format='0%')

p2 = figure(**plot_options, x_range=p1.x_range, title='Return vs Realized Vol',
            x_axis_label='return', y_axis_label='realize vol')
p2.hex('return', 'realized_vol', line_color="navy", fill_color="red", size=10, source=cds)
# formatting the x-axis and y-axis to percents
p2.xaxis.formatter = NumeralTickFormatter(format='0%') 
p2.yaxis.formatter = NumeralTickFormatter(format='0%')

p = gridplot([[p1,p2]])
show(p)

Loading BokehJS ...

### Hover Tool

Hover tools can be helpful for making large visualizations more readable.  Here we add one to our monthly returns bar chart.

In [6]:
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.plotting import reset_output
from bokeh.models import NumeralTickFormatter
import math
reset_output()

output_notebook()

source = ColumnDataSource(df_monthly)

# defining custom tool list
tools = 'box_zoom, reset'

# defining tool tip
tooltips = [
    ('month', '@year_month'),
    ('return', '@return{0.0%}'),
]

p = figure(width=1000, height=400, x_range=df_monthly['year_month'], tools=[tools], tooltips=tooltips,
           title='SPY Monthly Returns 2016Q1 to 2021Q3', x_axis_label='month', y_axis_label='monthly return')
p.vbar(x='year_month', bottom=0, top='return', color='blue', width=0.75, source=source)

# formatting the y-axis to percents
p.yaxis.formatter = NumeralTickFormatter(format='0%')

p.xaxis.major_label_orientation = math.pi/4

show(p)

Loading BokehJS ...

### Hover Tool + Interactive Legend

Here we add a hover tool to our asset allocation stacked area plot.

We also make the legend interactive to be able to hone in on specific allocation buckets.

In [7]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models import NumeralTickFormatter
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

output_notebook()

source = ColumnDataSource(df_asset_allocation)

p = figure(width=900, height=500, x_axis_type='datetime', title='Asset Allocation 2019-2020',
           x_axis_label='month', y_axis_label='allocation')

# choosing assets and colors to graph
assets = df_asset_allocation.drop(columns='trade_date').columns
num_assets = len(assets)
colors = bokeh.palettes.magma(num_assets) # choosing color palette

# adding glyphs
p.varea_stack(assets, x='trade_date', color=colors, source=source, legend_label=assets.to_list())
p.vline_stack(assets.to_list(), x='trade_date', color=colors, source=source,)

# defining tool tip
p.add_tools(HoverTool(
    tooltips = [
        ("Trade Date", "@trade_date{%F}"),
        ("VXX", "@VXX{0%}"),
        ("DBA", "@DBA{0%}"),
        ("USO", "@USO{0%}"),
        ("HYG", "@HYG{0%}"),
        ("TLT", "@TLT{0%}"),
        ("IWM", "@IWM{0%}"),
        ("SPY", "@SPY{0%}"),
    ],
    formatters={
        '@trade_date':'datetime', # use 'datetime' formatter for 'date' field
    },

))

# interactive legend
p.legend.click_policy='hide'

# reformatting 
p.yaxis.formatter = NumeralTickFormatter(format='0%')

# reversing ordering of legend to make it consistent with stacking order
p.legend[0].items.reverse()

# moving legend off of the graph
p.add_layout(p.legend[0], 'right')

show(p)

Loading BokehJS ...